In [1]:
import os
import numpy as np
from scipy.ndimage import rotate
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
sulcus = 'S.C.-sylv.'
side = 'L'

In [18]:
dataset = 'UkBioBank'

arrs = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/{side}skeleton.npy')
foldlabels = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/{side}label.npy')
distbottoms = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/{side}distbottom.npy')

In [19]:
dict_arrs = {'skeleton': arrs, 'foldlabel': foldlabels, 'distbottom': distbottoms}
#dict_arrs = {'skeleton': arrs}

In [2]:
# given two vectors, compute the rotations to map one to the other
def rotation(v1, v2):
    """
    Compute a matrix R that rotates v1 to align with v2.
    v1 and v2 must be length-3 1d numpy arrays.
    """
    # unit vectors
    u = v1 / np.linalg.norm(v1)
    Ru = v2 / np.linalg.norm(v2)
    # dimension of the space and identity
    dim = u.size
    I = np.identity(dim)
    # the cos angle between the vectors
    c = np.dot(u, Ru)
    # a small number
    eps = 1.0e-10
    if np.abs(c - 1.0) < eps:
        # same direction
        return I
    elif np.abs(c + 1.0) < eps:
        # opposite direction
        return -I
    else:
        # the cross product matrix of a vector to rotate around
        K = np.outer(Ru, u) - np.outer(u, Ru)
        # Rodrigues' formula
        return I + K + (K @ K) / (1 + c)
    
# remove zeros function
def trim_zeros(arr):
    """Returns a trimmed view of an n-D array excluding any outer
    regions which contain only zeros.
    """
    slices = tuple(slice(idx.min(), idx.max() + 1) for idx in np.nonzero(arr))
    return arr[slices], slices

In [3]:
a = [1, 0, 0]
b = [0.534, -0.293, 0.793] # TODO: HERE DEFINE THE NORMAL TO HULL # add a dictionnary with the right directions : iterate on each region !
b = b / np.linalg.norm(b)

rot = rotation(b,a)
print(rot)
print(np.dot(rot, b))

r = R.from_matrix(rot)
angles = r.as_euler('zyx', degrees=True)
print(angles)
angles = [-14.4, 28.75, 52.5] # TODO: is it the right order ??? shift from 1 to the left compared to what is given by normal to hull ?
# when angle is defined with morphologist, still need this reordering ?

[[ 0.53403899 -0.29302139  0.7930579 ]
 [ 0.29302139  0.9440291   0.15148437]
 [-0.7930579   0.15148437  0.59000988]]
[1.00000000e+00 1.26288993e-17 6.75722575e-17]
[ 28.75309824  52.47220192 -14.39959725]


all = np.stack((arrs, foldlabels, distbottoms), axis=4)

## check number of non zero voxels before
for idx, key in enumerate(keys):
    print(f'ARRAY TYPE : {key}')
    if key=='distbottom':
        cval=32501
    else:
        cval=0
    arrs = all[:,:,:,:,idx,0]
    print(f'Non zero voxels before rotation : {np.sum(arrs!=cval)}')


axes = [(1, 2), (1, 3), (2, 3)]

# rotate all modalities at once ## NOT FASTER ?
for ax, angle in zip(axes, angles):
    rot_all = rotate(all,
                     angle=angle,
                     axes=ax,
                     order=0,
                     reshape=True,
                     mode='constant',
                     cval=cval)

## trim zeros
# first set to 0 the 32500+ values in distbottom
for idx, key in enumerate(keys):
    if  key=='distbottom':
        distbottoms = rot_all[:,:,:,:,idx,:]
        distbottoms[distbottoms==0]=-1
        distbottoms[distbottoms>=32500]=0
        rot_all[:,:,:,:,idx,:]=distbottoms
rot_all_trimmed = trim_zeros(rot_all)

for idx, key in enumerate(keys):
    arr = rot_all_trimmed[:,:,:,:,idx,:]
    print(f'Non zero voxels after rotation : {np.sum(arr!=0)}')
    if key=='distbottom':
        arr[arr==0]=32501
        arr[arr==-1]=0
        rot_all_trimmed[:,:,:,:,idx,:]=arr

save_dir = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/S.C.-sylv./mask/'
np.save(os.path.join(save_dir, f'{side}skeleton_rot.npy'), rot_all_trimmed[:,:,:,:,0,:])
np.save(os.path.join(save_dir, f'{side}label_rot.npy'), rot_all_trimmed[:,:,:,:,1,:])
np.save(os.path.join(save_dir, f'{side}distbottom_rot.npy'), rot_all_trimmed[:,:,:,:,2,:])

In [14]:
dict_rot_arrs = {}

axes = [(0, 1), (0, 2), (1, 2)]
for key, arrs in dict_arrs.items():
    print(f'ARRAY TYPE : {key}')
    if key=='distbottom':
        cval=32501
    else:
        cval=0
    arr_list = []
    print(f'Non zero voxels before rotation : {np.sum(arrs!=cval)}')
    for arr in tqdm(arrs):
        arr = arr[:,:,:,0]
        rot_arr = np.copy(arr)
        for ax, angle in zip(axes, angles):
            rot_arr = rotate(rot_arr,
                                angle=angle,
                                axes=ax,
                                order=0,
                                reshape=True,
                                mode='constant',
                                cval=cval)
        arr_list.append(rot_arr)

    rot_arrs = np.stack(arr_list)
    print(f'Non zero voxels after rotation : {np.sum(rot_arrs!=cval)}')
    print(f'Dimension after rotation : {rot_arrs.shape}')
    if key=='distbottom':
        # set 0 to vx to remove (temporarily)
        rot_arrs[rot_arrs==0]=-1
        rot_arrs[rot_arrs>=32500]=0
    rot_arrs_trimmed, slices = trim_zeros(rot_arrs)
    print(slices)
    if key=='distbottom':
        rot_arrs_trimmed[rot_arrs_trimmed==0]=32501
        rot_arrs_trimmed[rot_arrs_trimmed==-1]=0
    print(f'Dimension after trimming : {rot_arrs_trimmed.shape}')
    # volume ration should be < 1 !
    print(f"volume ratio after / before rotation : {np.prod(rot_arrs_trimmed.shape[1:]) / np.prod(arrs.shape[1:])}")
    # are topological values affected ?
    #print(np.unique(arrs, return_counts=True))
    #print(np.unique(rot_arrs_trimmed, return_counts=True))
    rot_arrs_trimmed = np.expand_dims(rot_arrs_trimmed, axis=-1)
    dict_rot_arrs[key]= rot_arrs_trimmed

  1%|          | 6/1114 [00:00<00:19, 57.69it/s]

ARRAY TYPE : skeleton
Non zero voxels before rotation : 1441847


100%|██████████| 1114/1114 [00:15<00:00, 71.53it/s]


Non zero voxels after rotation : 1437203
Dimension after rotation : (1114, 38, 55, 44)


  1%|          | 7/1114 [00:00<00:17, 64.59it/s]

(slice(0, 1114, None), slice(0, 37, None), slice(3, 53, None), slice(7, 39, None))
Dimension after trimming : (1114, 37, 50, 32)
volume ratio after / before rotation : 0.8831602816565223
ARRAY TYPE : foldlabel
Non zero voxels before rotation : 1441847


100%|██████████| 1114/1114 [00:15<00:00, 71.25it/s]


Non zero voxels after rotation : 1437203
Dimension after rotation : (1114, 38, 55, 44)
(slice(0, 1114, None), slice(0, 37, None), slice(3, 53, None), slice(7, 39, None))
Dimension after trimming : (1114, 37, 50, 32)
volume ratio after / before rotation : 0.8831602816565223


In [23]:
save_dir = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/'
np.save(os.path.join(save_dir, f'{side}skeleton_rot.npy'), dict_rot_arrs['skeleton'])
np.save(os.path.join(save_dir, f'{side}label_rot.npy'), dict_rot_arrs['foldlabel'])
np.save(os.path.join(save_dir, f'{side}distbottom_rot.npy'), dict_rot_arrs['distbottom'])

In [20]:
dict_rot_arrs['skeleton'].shape

(1114, 37, 52, 31, 1)

In [21]:
## hcp
dataset = 'hcp'

arrs = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/{side}skeleton.npy')
foldlabels = np.load(f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/{side}label.npy')

In [22]:
dict_arrs = {'skeleton': arrs}

In [29]:
#angles = [-100.39277232, -142.04451126, -44.85192686] # 0.82
#angles = [-142.04451126,  -44.85192686, -100.39277232] #0.88
#angles = [-44.85192686, -100.39277232, -142.04451126] # 0.89
angles = [-56.21228385,   4.1782546,    2.23226287] # 0.93, ai-je inversé les angles ? le masque a-t-il une forme bizarre ?

In [30]:
dict_rot_arrs = {}

axes = [(0, 1), (0, 2), (1, 2)]
for key, arrs in dict_arrs.items():
    print(f'ARRAY TYPE : {key}')
    if key=='distbottom':
        cval=32501
    else:
        cval=0
    arr_list = []
    print(f'Non zero voxels before rotation : {np.sum(arrs!=cval)}')
    for arr in tqdm(arrs):
        arr = arr[:,:,:,0]
        rot_arr = np.copy(arr)
        for ax, angle in zip(axes, angles):
            rot_arr = rotate(rot_arr,
                                angle=angle,
                                axes=ax,
                                order=0,
                                reshape=True,
                                mode='constant',
                                cval=cval)
        arr_list.append(rot_arr)

    rot_arrs = np.stack(arr_list)
    print(f'Non zero voxels after rotation : {np.sum(rot_arrs!=cval)}')
    print(f'Dimension after rotation : {rot_arrs.shape}')
    if key=='distbottom':
        # set 0 to vx to remove (temporarily)
        rot_arrs[rot_arrs==0]=-1
        rot_arrs[rot_arrs>=32500]=0
    rot_arrs_trimmed, slices = trim_zeros(rot_arrs)
    #rot_arrs_trimmed = rot_arrs[slices] ## use slices instead of trim_zeros !
    if key=='distbottom':
        rot_arrs_trimmed[rot_arrs_trimmed==0]=32501
        rot_arrs_trimmed[rot_arrs_trimmed==-1]=0
    print(f'Dimension after trimming : {rot_arrs_trimmed.shape}')
    # volume ration should be < 1 !
    print(f"volume ratio after / before rotation : {np.prod(rot_arrs_trimmed.shape[1:]) / np.prod(arrs.shape[1:])}")
    # are topological values affected ?
    print(np.unique(arrs, return_counts=True))
    print(np.unique(rot_arrs_trimmed, return_counts=True))
    rot_arrs_trimmed = np.expand_dims(rot_arrs_trimmed, axis=-1)
    dict_rot_arrs[key]= rot_arrs_trimmed

  1%|          | 7/1114 [00:00<00:16, 65.53it/s]

ARRAY TYPE : skeleton
Non zero voxels before rotation : 1441847


100%|██████████| 1114/1114 [00:13<00:00, 81.18it/s]


Non zero voxels after rotation : 1438604
Dimension after rotation : (1114, 38, 38, 50)
Dimension after trimming : (1114, 37, 36, 47)
volume ratio after / before rotation : 0.9339419978517723
(array([ 0, 10, 30, 35, 40, 50, 60, 70, 80, 90], dtype=int16), array([73231801,       52,   204665,   179839,     8988,      131,
        1018623,      458,    29090,        1]))
(array([ 0, 10, 30, 35, 40, 50, 60, 70, 80, 90], dtype=int16), array([68302252,       52,   204365,   178778,     8938,      132,
        1016829,      458,    29051,        1]))


In [11]:
save_dir = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{sulcus}/mask/'
np.save(os.path.join(save_dir, f'{side}skeleton_rot.npy'), dict_rot_arrs['skeleton'])
np.save(os.path.join(save_dir, f'{side}label_rot.npy'), dict_rot_arrs['foldlabel'])